In [ ]:
from glob import glob

import numpy as np
import h5py

from scipy.signal import argrelextrema
from matplotlib   import pyplot as plt

In [ ]:
alpha = 16*((np.arange(1024)+0.5)/1024-0.5)

def da(f):
    k  = f[:,1]
    ki = k[-1,1:]
    kf = k[ 0,1:]
    
    fi = np.einsum('ijk,ijk->jk', kf, ki)
    ff = np.einsum('ijk,ijk->jk', kf, kf)
    ii = np.einsum('ijk,ijk->jk', ki, ki)
    
    return np.arccos(fi / np.sqrt(ff * ii))

def argmax(d):
    am = argrelextrema(d, np.greater)[0]
    am = np.array([a for a in am if max(d[a+1]-d[a], d[a]-d[a-1]) > 0.02])
    return am

def argmin(d):
    am = argrelextrema(d, np.less)[0]
    am = np.array([a for a in am if min(d[a+1]-d[a], d[a]-d[a-1]) <-0.02])
    return am

def unfold(d):
#     aM = argmax(deflection)
#     aR = np.where(deflection[aM[:len(aM)//2]] > 3)[0]
#     if len(aR) >= 2:
#         deflection[aM[aR[0]]+1:aM[aR[-1]]] = 2*np.pi - deflection[aM[aR[0]]+1:aM[aR[-1]]]

#     aM = argmax(deflection)
#     aR = np.where(deflection[aM[:len(aM)//2]] > 6)[0]
#     if len(aR) >= 2:
#         deflection[aM[aR[0]]+1:aM[aR[-1]]] = 4*np.pi - deflection[aM[aR[0]]+1:aM[aR[-1]]]

#     aM = argmax(deflection)
#     aR = np.where(deflection[aM[:len(aM)//2]] > 9)[0]
#     if len(aR) >= 2:
#         deflection[aM[aR[0]]+1:aM[aR[-1]]] = 6*np.pi - deflection[aM[aR[0]]+1:aM[aR[-1]]]

#     aM = argmax(deflection)
#     aR = np.where(deflection[aM[:len(aM)//2]] > 12)[0]
#     if len(aR) >= 2:
#         deflection[aM[aR[0]]+1:aM[aR[-1]]] = 8*np.pi - deflection[aM[aR[0]]+1:aM[aR[-1]]]

#     am = argmin(deflection)    
#     try:
#         deflection[am[0]:am[-1]] *= -1
#     except:
#         pass
    return d

In [ ]:
with h5py.File('image_a2.00_i30.h5') as h:
    f = h['f'][:]

t = f[0,0,0]
x = f[0,0,1]
y = f[0,0,2]
z = f[0,0,3]
r = np.sqrt(x*x + y*y + z*z)

print(f.shape)
print(t.shape)

In [ ]:
print(np.nanmin(t), np.nanmax(t))
print(np.nanmin(r), np.nanmax(r))

In [ ]:
aspins = 2**np.linspace(0,3,num=13)[1:]

for a in aspins:
    for i in [0]:#range(0,91,15):
        file = f'image_a{a:.02f}_i{i}.h5'
        
        try:
            with h5py.File(file) as h:
                f = h['f'][:]
                print(file)        
        except:
            continue
    
        fig, ax = plt.subplots(1,1,figsize=(8,8))
        deflection = unfold(da(f))
        ax.imshow(
            deflection.T, origin='lower', extent=[-8,8,-8,8], 
            vmin=0, vmax=np.pi, cmap='seismic',
        )
        ax.set_title(f'aspin={a:.02f}, inc={i}')
    
        fig.savefig(file.replace('h5', 'png'), dpi=300, bbox_inches='tight')
        plt.close(fig)